In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, balanced_accuracy_score,
                           confusion_matrix, classification_report)
import lightgbm as lgb
from imblearn.combine import SMOTEENN
import warnings
warnings.filterwarnings('ignore')


def load_and_preprocess():
    """Load and preprocess the data"""
    print("Loading and preprocessing data...")
    
    # Load training and testing datasets
    train_data = pd.read_csv('../Dataset/cleaned_customer_churn_dataset_training.csv')
    test_data = pd.read_csv('../Dataset/cleaned_customer_churn_dataset_testing.csv')
    
    # Separate features and target
    X_train = train_data.drop(['CustomerID', 'Churn'], axis=1)
    y_train = train_data['Churn']
    X_test = test_data.drop(['CustomerID', 'Churn'], axis=1)
    y_test = test_data['Churn']
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Handle class imbalance using SMOTEENN
    smote_enn = SMOTEENN(sampling_strategy=0.8, random_state=42)
    X_train_balanced, y_train_balanced = smote_enn.fit_resample(X_train_scaled, y_train)
    
    print("Data shapes:")
    print(f"Training data: {X_train_balanced.shape}")
    print(f"Testing data: {X_test_scaled.shape}")
    print("\nClass distribution in training set after balancing:")
    print(pd.Series(y_train_balanced).value_counts(normalize=True))
    
    return X_train_balanced, X_test_scaled, y_train_balanced, y_test, X_train.columns

def train_lightgbm(X_train, y_train):
    """Train LightGBM model with optimized parameters"""
    print("\nTraining LightGBM model...")
    
    model = lgb.LGBMClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        num_leaves=20,
        subsample=0.7,
        colsample_bytree=0.7,
        min_child_samples=20,
        verbose=-1
    )
    
    # Train the model
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test, feature_names):
    """Evaluate the model and display results"""
    # Make predictions
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Find optimal threshold
    thresholds = np.arange(0.3, 0.7, 0.05)
    best_threshold = 0.5
    best_f1 = 0
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y_test, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    
    # Final predictions with optimal threshold
    y_pred = (y_pred_proba >= best_threshold).astype(int)
    
    # Print metrics
    print("\nModel Performance Metrics:")
    print(f"Optimal threshold: {best_threshold:.3f}")
    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred):.3f}")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    
    # Classification report
    print("\nDetailed Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Feature importance plot
    plt.figure(figsize=(12, 6))
    importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    sns.barplot(data=importance.head(10), x='Importance', y='Feature')
    plt.title('Top 10 Most Important Features')
    plt.tight_layout()
    plt.show()
    
    # Print top 5 features
    print("\nTop 5 Most Important Features:")
    print(importance.head().to_string(index=False))

def main():
    # Load and preprocess data
    X_train, X_test, y_train, y_test, feature_names = load_and_preprocess()
    
    # Train model
    model = train_lightgbm(X_train, y_train)
    
    # Evaluate model
    evaluate_model(model, X_test, y_test, feature_names)

if __name__ == "__main__":
    main()

Loading and preprocessing data...
